In [1]:
mlist = [
    "distilbert-base-uncased-finetuned-sst-2-english",
    "ProsusAI/finbert", 
    "j-hartmann/emotion-english-distilroberta-base",
    
]

import pandas as pd
from datasets import Dataset, DatasetDict
import mlflow


df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')


train = Dataset.from_pandas(df_train)
test = Dataset.from_pandas(df_test)


dataset = DatasetDict()

dataset['train'] = train
dataset['test'] = test

for x in mlist:
  # %%
  print(x)
  # %%
  from transformers import AutoTokenizer
  model_name = x

  tokenizer = AutoTokenizer.from_pretrained(model_name)
  # %%

  def preprocess_function(examples):
      return tokenizer(examples["text"], truncation=True)

  tokenized_datasets = dataset.map(preprocess_function, batched=True)
  # %%
  from transformers import DataCollatorWithPadding

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
  # %%
  from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

  model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                            num_labels=8,
                                                            ignore_mismatched_sizes=True)
  # %%
  import numpy as np
  from datasets import load_metric
  metric = load_metric("accuracy")
  #%%
  def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
  # %%
  num_epochs = 2

  args = TrainingArguments(
      output_dir ='./results',          
      num_train_epochs = num_epochs,
      per_device_train_batch_size = 16,   
      per_device_eval_batch_size = 16,   
      weight_decay = 0.01,               
      learning_rate = 2e-5,             
      metric_for_best_model = 'accuracy',    
      evaluation_strategy = "epoch",
      )


  trainer = Trainer(
      model,
      args,
      train_dataset=tokenized_datasets['train'],
      eval_dataset=tokenized_datasets['test'],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics
  )  
  mlflow.autolog()
  trainer.train()
  mlflow.log_param("model",x)

distilbert-base-uncased-finetuned-sst-2-english


  0%|          | 0/41 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([8, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([8]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/d0/_y2f7ycs76b8ryly86zqzgw00000gn/T/ipykernel_55633/1772648670.py:53: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
2023/01/08 00:24:35 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
The following columns in the trai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/5120 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 